# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

Please make sure if you are running this notebook in the workspace that you have chosen GPU rather than CPU mode.

In [ ]:
import torch
from torchvision import datasets, transforms
import torchvision.models as models
from torch import optim
from torch import nn
import torch.nn.functional as F

from collections import OrderedDict
import json
import time
import numpy as np
import matplotlib.pyplot as plt
import copy
from PIL import Image

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [ ]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
dataset_directories = {'train' : train_dir, 'test' : test_dir, 'validation': valid_dir}
all_datasets = ('train', 'test', 'validation')

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

In [ ]:
# Transforms: only resize/crop for training and validation
data_transforms = {
    'train' : transforms.Compose([transforms.RandomRotation(40),
                                  transforms.RandomResizedCrop(224),
                                  transforms.RandomHorizontalFlip(0.55),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean, std)]),
    
    'validation' : transforms.Compose([transforms.Resize(256),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean,std)]),
    
    'test' : transforms.Compose([transforms.Resize(256),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean,std)])
}

In [ ]:
# Loading the datasets with ImageFolder
image_datasets = {x : datasets.ImageFolder(dataset_directories[x], transform=data_transforms[x]) for x in all_datasets}

# Creating dictionary of DataLoaders
dataloaders = {x : torch.utils.data.DataLoader(image_datasets[x], batch_size=32) for x in all_datasets}

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    
print("Number of labels: {}".format(len(cat_to_name)))

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

In [ ]:
# Downloading pretrained VGG model
model = models.vgg19(pretrained=True)

# Freezing the weights
for param in model.parameters():
    param.requires_grad = False

# Creating new classifier
# Output layer: 102 neurons for the labels
classifier = nn.Sequential(OrderedDict([
                          ('fully_connected_1', nn.Linear(25088, 4096)),
                          ('relu', nn.ReLU()),
                          ('dropout', nn.Dropout(0.5)),
                          ('fully_connected_2', nn.Linear(4096, 102)),
                          ('output_layer', nn.LogSoftmax(dim=1))
                          ]))

model.classifier = classifier

In [ ]:
# Criterion and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

In [ ]:
# Setting parameters for training
epochs = 10
steps = 0
print_every = 40
datasets = ('train','validation')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
model.to(device);

In [ ]:
def print_separated_message(*args):
    """ Beautifying message with two lines """
    print("=" * 60)
    for arg in args:
        print(arg)
    print("=" * 60)

In [ ]:
def train_model(model, criteria, optimizer, number_of_epochs, device):
    """This method trains the model, and prints training/validation losses and accuracies."""
    
    start_time = time.time()
    model.to(device)

    training_phases = ('train', 'validation')
    best_state_dict = model.state_dict()
    best_accuracy = 0

    for epoch in range(number_of_epochs):
        print_separated_message('Epoch {}/{}'.format(epoch + 1, number_of_epochs))
        for training_phase in training_phases:
            dataset_size = len(image_datasets[training_phase])
            if training_phase == 'train':
                model.train(True)
            else:
                model.train(False)
            running_loss = 0.0
            running_correct = 0

            for inputs, labels in dataloaders[training_phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(training_phase == 'train'):
                    outputs = model(inputs)
                    temp, predictions = torch.max(outputs, 1)
                    loss = criteria(outputs, labels)
                    if training_phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_correct += torch.sum(predictions == labels.data)
            epoch_loss = running_loss / dataset_size
            epoch_accuracy = running_correct.double() / dataset_size

            print('Phase: {}, loss: {:.3f}, accuracy: {:.1f} %'.format(training_phase, epoch_loss, epoch_accuracy * 100.0))

            if training_phase == 'validation' and epoch_accuracy > best_accuracy:
                best_accuracy = epoch_accuracy
                best_state_dict = copy.deepcopy(model.state_dict())

    elapsed_time = time.time() - start_time

    print_separated_message('Training took {:.0f} minutes {:.0f} seconds'.format(elapsed_time // 60, elapsed_time % 60))
    print_separated_message('Best validation accuracy: {:1f}%'.format(best_accuracy * 100.0))

    model.load_state_dict(best_state_dict)
    return model

In [ ]:
# Training the model
trained_model = train_model(model, criterion, optimizer, epochs, device)

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
def test_accuracy(model_to_test, testloader, device):   
    """Testing model, printing accuracy and calculating elapsed time"""
    
    print_separated_message("Testing model accuracy")
    start_time = time.time()
    model_to_test.eval()
    model_to_test.to(device)
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model_to_test(images)
            temp, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print_separated_message('Accuracy of the model: %d %%' % (100 * correct / total))
    
    elapsed_time = time.time() - start_time
    print_separated_message("Testing accuracy took {:05.2f} minutes".format(elapsed_time / 60))

In [ ]:
test_accuracy(trained_model, dataloaders['test'], device)

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# Saving checkpoint
def save_checkpoint(model_to_save, dataset, name='classifier'):
    """
    Saves model type, state_dict and class_to_idx of model into name.pth file.
    Default file name is classifier.pth
    """
    filename = name + '.pth'
    model_to_save.class_to_idx = dataset.class_to_idx
    model_to_save.cpu()
    
    torch.save({'model type': 'vgg19', 
                'state_dict': model_to_save.state_dict(), 
                'class_to_idx': model_to_save.class_to_idx}, filename)

In [ ]:
save_checkpoint(model, image_datasets['train'])

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
def load_model(checkpoint):
    """Loads checkpoint and returns the model"""
    
    checkpoint = torch.load(checkpoint)
    model = models.vgg19(pretrained=True)
    for parameter in model.parameters():
        parameter.require_grad = False
    
    model.class_to_idx = checkpoint['class_to_idx']
    
    classifier = nn.Sequential(OrderedDict([
                          ('fully_connected_1', nn.Linear(25088, 4096)),
                          ('relu', nn.ReLU()),
                          ('dropout', nn.Dropout(0.5)),
                          ('fully_connected_2', nn.Linear(4096, 102)),
                          ('output_layer', nn.LogSoftmax(dim=1))
                          ]))
    
    model.classifier = classifier
    model.load_state_dict(checkpoint['state_dict'])
    return model
        

In [ ]:
loaded_model = load_model('classifier.pth')

In [ ]:
test_accuracy(loaded_model, dataloaders['test'], 'cpu')

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image_path):
    ''' 
    Scales, crops, and normalizes a PIL image for a PyTorch.     
    model, returns an Numpy array.
    Transforming is done with PyTorch transforms library.
    Output tensor is converted into Numpy array.
    '''
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    
    pil_image = Image.open(image_path)
    
    trans = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ])
    
    tensor_image = trans(pil_image)
    return tensor_image.numpy()

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    if title:
        plt.title(title)
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    ax.imshow(image)
    
    return ax.axis('off')

In [ ]:
test_image = test_dir + '/43/image_02365.jpg'
title = cat_to_name['43'].upper()
img = process_image(test_image)

In [ ]:
imshow(img, title=title)

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, device, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    model.to(device)
    numpy_array = process_image(image_path)
    
    # Converting numpy array back to torch tensor
    tensor = torch.tensor(numpy_array).float()
    tensor = tensor.unsqueeze_(0)

    with torch.no_grad():
        output = model.forward(tensor.to(device))
    probabilities = F.softmax(output.data,dim=1)
    result =  probabilities.topk(topk)
    
    # Converting softmax results to numpy arrays
    # Returning 1D arrays of probabilities/classes
    probs = result[0].cpu().detach().numpy()
    classes = result[1].cpu().detach().numpy()
    return probs[0], classes[0]
    

In [ ]:
probs, classes = predict(test_image, loaded_model, device)

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
def sanity_check(image, probs, classes):
    """Printing probabilities of the top 5 classes, and displaying image in question.
    """
    labels = [cat_to_name[str(x)].upper() for x in classes]
    
    img = process_image(test_image)
    
    plt.figure(figsize = (6,10))
    ax = plt.subplot(2,1,1)
    imshow(img, ax=ax, title = title);
    
    chart_color = sb.color_paleatte()[0]
    
    plt.subplot(2,1,2)
    sb.barplot(x=probs, y=labels, color=chart_color);
    plt.show()

In [ ]:
sanity_check(test_image, probs, classes)